<a href="https://colab.research.google.com/github/WinuWijegunawardhana98/Integrated-Healthcare-Management-System-for-Vulnerable-Populations/blob/winodya/converting_image_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install pytesseract pillow spacy
!python -m spacy download en_core_web_sm
!pip install transformers



import pandas as pd
from sklearn.model_selection import train_test_split
import os
from PIL import Image
import spacy
import pytesseract
import cv2
import numpy as np

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 24.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [47]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
folder_path = '/content/drive/MyDrive/Dataset/New folder - winu'
for filename in os.listdir(folder_path):
    print(filename)
if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        print("Processing {file_path}")

final.csv
README
diabetes-data.tar.Z
Index
data.yaml
drug adherence
Drug types by the name
Drug Vision(images of the tablets)
Doctor’s Handwritten Prescription BD dataset
Drug Vision
prescription data (images)


In [37]:
print(data.head())

   Unnamed: 0                    disease                          drug
0           0  Alkylating Agent Cystitis            sodium bicarbonate
1           1  Alkylating Agent Cystitis  citric acid / sodium citrate
2           2       Abdominal Distension                   bethanechol
3           3       Abdominal Distension                      pamabrom
4           4       Abdominal Distension                   bethanechol


In [38]:
# Example: Apply OCR to all images in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        file_path = os.path.join(folder_path, filename)
        print(f"Processing {filename}...")

        # Open the image and extract text
        image = Image.open(file_path)
        text = pytesseract.image_to_string(image)
        print(f"Extracted Text from {filename}:")
        print(text)

In [39]:
# Process all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        print(f"Reading {filename}...")
        data = pd.read_csv(file_path)
        print(f"First 5 rows of {filename}:")
        print(data.head())

Reading final.csv...
First 5 rows of final.csv:
   Unnamed: 0                    disease                          drug
0           0  Alkylating Agent Cystitis            sodium bicarbonate
1           1  Alkylating Agent Cystitis  citric acid / sodium citrate
2           2       Abdominal Distension                   bethanechol
3           3       Abdominal Distension                      pamabrom
4           4       Abdominal Distension                   bethanechol


In [40]:
# Load SpaCy model
nlp = spacy.load('en_core_web_sm')

# Process each image
for filename in os.listdir(folder_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        file_path = os.path.join(folder_path, filename)
        print(f"Processing {filename}...")

        # OCR Step
        image = Image.open(file_path)
        text = pytesseract.image_to_string(image)

        # NLP Step
        doc = nlp(text)
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        print(f"Extracted Entities from {filename}:")
        print(entities)

In [41]:
output = []

# Collect results
for filename in os.listdir(folder_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        file_path = os.path.join(folder_path, filename)
        image = Image.open(file_path)
        text = pytesseract.image_to_string(image)
        doc = nlp(text)
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        output.append({'filename': filename, 'text': text, 'entities': entities})

# Save to CSV
output_df = pd.DataFrame(output)
output_df.to_csv('/content/drive/My Drive/processed_results.csv', index=False)
print("Results saved to 'processed_results.csv'.")


Results saved to 'processed_results.csv'.


In [26]:
# Replace with the path to your folder in Google Drive
folder_path = '/content/drive/MyDrive/Dataset/New folder - winu/Doctor’s Handwritten Prescription BD dataset'

# Verify the files in the folder
print("Files in the folder:")
for filename in os.listdir(folder_path):
    print(filename)

Files in the folder:
Validation
Training


In [42]:
# Load SpaCy NLP model
nlp = spacy.load('en_core_web_sm')

# Initialize a list to store the results
results = []

# Process each image in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
        file_path = os.path.join(folder_path, filename)
        print(f"Processing {filename}...")

        # Step 1: OCR - Extract text from image
        image = Image.open(file_path)
        text = pytesseract.image_to_string(image)
        print(f"Extracted Text from {filename}:\n{text}\n")

        # Step 2: NLP - Process the extracted text
        doc = nlp(text)
        entities = [(ent.text, ent.label_) for ent in doc.ents]  # Extract entities

        # Store the results
        results.append({
            'filename': filename,
            'text': text,
            'entities': entities
        })

In [43]:
# Convert results to a DataFrame
output_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file in Google Drive
output_file_path = '/content/drive/My Drive/ocr_nlp_results.csv'
output_df.to_csv(output_file_path, index=False)
print(f"Results saved to {output_file_path}")

Results saved to /content/drive/My Drive/ocr_nlp_results.csv


In [44]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer

# Define the label map (replace with your actual labels)
label_map = {'Disease': 0, 'Medicine': 1, 'Dosage': 2}

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(label_map))

# Prepare data for training
# Convert text into tokens and align with labels

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
import pytesseract
import spacy

def extract_text(image_path):
    """
    Extracts text from an image using pytesseract.
    """
    try:
        image = Image.open(image_path)
        text = pytesseract.image_to_string(image)
        return text
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return ""  # Return empty string on error

def extract_entities(text):
    """
    Extracts entities from text using spaCy.
    """
    nlp = spacy.load('en_core_web_sm')  # Load spaCy model
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

def process_prescription(image_path):
    """
    Processes a prescription image to extract text and entities.
    """
    text = extract_text(image_path)  # OCR Step
    entities = extract_entities(text)  # NLP Step
    return entities


